In [31]:
from elasticsearch import Elasticsearch
import tiktoken

In [2]:
es_client = Elasticsearch("http://localhost:9200")

In [3]:
es_client.info()

ObjectApiResponse({'name': '41bc9c3df210', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Ao20Kx-5TNGlA4ZhAPqlBw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [4]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [6]:
from tqdm.auto import tqdm

/Users/gbotemi/miniconda3/envs/llm-zoomcamp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
for doc in tqdm(documents):
	es_client.index(index=index_name, document=doc,)

100%|██████████| 948/948 [00:03<00:00, 304.56it/s]


In [8]:
query = "How do I execute a command in a running docker container?"

In [15]:
query_settings = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [16]:
response = es_client.search(index=index_name, body=query_settings)

In [17]:
response['hits']['hits'][2]

{'_index': 'course-questions',
 '_id': 'G6XcaZABzWtjS4pEmOmC',
 '_score': 49.938507,
 '_source': {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from a different folder into docker container’s working directory?',
  'course': 'machine-learning-zoomcamp'}}

In [12]:
response['hits']['hits'][0]["_source"]["text"]	

'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for root:\nServer: PostgreSQL 16.1 (Debian 16.1-1.pgdg120+1)\nVersion: 4.0.1\nHome: http://pgcli.com\nroot@pg-database:ny_taxi> \\dt\n+--------+------------------+-------+-------+\n| Schema | Name             | Type  | Owner |\n|--------+------------------+-------+-------|\n| public | yellow_taxi_data | table | root  |\n+--------+------------------+-------+-------+\nSELECT 1\nTime: 0.009s\nroot@pg-database:ny_taxi>'

In [20]:
results_docs = []

for hit in response['hits']['hits']:
	source = hit["_source"]
	results_docs.append(source)

In [22]:
results_docs[0]

{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
 'section': '5. Deploying Machine Learning Models',
 'question': 'How do I debug a docker container?',
 'course': 'machine-learning-zoomcamp'}

In [18]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [19]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [23]:
prompt_template

"You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: {question}\n\nCONTEXT:\n{context}"

In [24]:
def search_elastic(query):
	query_settings = {
		"size": 5,
		"query": {
			"bool": {
				"must": {
					"multi_match": {
						"query": query,
						"fields": ["question^4", "text"],
						"type": "best_fields"
					}
				},
				"filter": {
					"term": {
						"course": "machine-learning-zoomcamp"
					}
				}
			}
		}
	}
	
	index_name = "course-questions"
	response = es_client.search(index=index_name, body=query_settings)

	results_docs = []

	for hit in response['hits']['hits']:
		source = hit["_source"]
		results_docs.append(source)

	return results_docs

In [25]:
def build_prompt(query, search_results):
	prompt_template = """
		You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
		Use only the facts from the CONTEXT when answering the QUESTION.

		QUESTION: {question}

		CONTEXT:
		{context}
		""".strip()
	
	context = ""
	for doc in search_results:
		context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
	prompt = prompt_template.format(question=query, context=context).strip()
	return prompt

In [26]:
def rag(query):
    search_result = search_elastic(query)
    prompt = build_prompt(query, search_result)
    # answer = llm(prompt)
    return prompt

In [36]:
prompt = rag("How do I execute a command in a running docker container?")

In [52]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [54]:
num_tokens_from_string(prompt, "gpt-4o")

684